In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "5호모델드랍down"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'5호모델드랍down_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="최대주주변경카운트", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '불성실공시법인지정카운트', '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+최대주주변경카운트"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [17]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트"
                                 ,plus_two="상호변경카운트+업종변경카운트", return_origianl=False)
concat_together9["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액"
                                 ,plus_two="타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액", return_origianl=False)
concat_together10["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       'BW금액+CB금액+유상증자금액', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [19]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액"
                                 ,plus_two="BW금액+CB금액+유상증자금액", return_origianl=False)
print(concat_together11["2012-01"].columns.shape)
concat_together11["2012-01"].columns

(14,)


Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액',
       '경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [20]:
drop_col_top_y = ["매출총이익","총자본","영업이익","매출채권"] # q와 동일
#drop_col_top_q = ["당기순이익/총자본","영업이익/총자본","총자본","매출총이익/총자본"]

drop_col_down_y = ["차입금의존도",
                   "최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액+타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액+BW금액+CB금액+유상증자금액",
                   "최대주주비율_변동성","결산기간_거래량증가"]

divide_data2 = tools.dict_data_drop(concat_together11.copy(), drop_col=drop_col_down_y, dropna=False)

In [21]:
drop_togehter = tools.dict_data_drop(divide_data2.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '총자본',
       '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [22]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 12.81it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 4,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=4)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2014-01': {'num_of_cluseer': 3,
  'random_state': 3,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=3)},
 '2015-01': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2016-01': {'num_of_cluseer': 3,
  'random_state': 6,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=6)},
 '2017-01': {'num_of_cluseer': 3,
  'random_state': 10,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=10)},
 '2018-01': {'num_of_cluseer': 3,
  'random_state': 28,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=28)},
 '2019-01': {'num_of_cluseer': 3,
  'random_state': 63,
  'model': KMeans(max_iter=500, n_clu

## 모델기록


In [23]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

792.0
462.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,544.0,566.0,496.0,630.0,629.0,531.0,719.0,670.0,740.0,792.0,774.0
1,468.0,596.0,585.0,525.0,540.0,629.0,575.0,567.0,537.0,540.0,559.0
2,616.0,462.0,572.0,557.0,591.0,677.0,611.0,716.0,734.0,728.0,735.0
기업수,1628.0,1624.0,1653.0,1712.0,1760.0,1837.0,1905.0,1953.0,2011.0,2060.0,2068.0


In [24]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [25]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [26]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [27]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8720304599030895
1.0


0.10387385421213444

In [28]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

매출총이익                                                       0.181623
총자본                                                         0.157069
매출채권                                                        0.135357
판매 및 관리비                                                    0.129933
영업이익                                                        0.123468
당기순이익                                                       0.090193
영업현금흐름                                                      0.080382
최대주주비율                                                      0.057887
결산기간_주가변동률                                                  0.025607
경영권분쟁소송+최대주주변경카운트+불성실공시법인지정카운트+조회공시요구카운트+상호변경카운트+업종변경카운트    0.018481
dtype: float64

In [29]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2013-01예측,2014-01예측,2015-01예측,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.828571,0.790698,0.023810,0.069767,0.211538,0.046512,0.078652,0.876404,0.090909
1,0.085714,0.162791,0.119048,0.023256,0.038462,0.825581,0.000000,0.011236,0.030303
2,0.085714,0.046512,0.857143,0.906977,0.750000,0.127907,0.921348,0.112360,0.878788


In [30]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2013-01예측,2014-01예측,2015-01예측,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.828571,0.790698,0.023810,0.069767,0.211538,0.046512,0.078652,0.876404,0.090909
1,0.085714,0.162791,0.119048,0.023256,0.038462,0.825581,0.000000,0.011236,0.030303
2,0.085714,0.046512,0.857143,0.906977,0.750000,0.127907,0.921348,0.112360,0.878788


In [31]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [32]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.754194,0.218877,-0.053528,1,2.5
A000040,-0.287095,0.100616,0.617085,2,2.5
A000050,0.783271,-0.129182,0.216164,1,2.5
A000070,1.211785,0.068399,-0.081046,1,2.5
A000080,1.168810,-0.046077,0.074102,1,2.5
...,...,...,...,...,...
A377300,0.526481,-0.137822,0.288687,2,2.5
A381970,0.416016,-0.316445,0.244913,2,2.5
A383220,1.009791,-0.235918,-0.149569,1,2.5
A383310,-0.142783,0.030774,-0.415929,2,2.5
